In [1]:
import numpy as np
import time
from collections import namedtuple
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import qiskit as q
import qiskit.tools.jupyter
from qiskit.pulse import pulse_lib as _  # only want to call pulse_lib as q.pulse.pulse_lib

In [3]:
import pulse_compiler_helper_fns

In [4]:
q.IBMQ.load_account() # Load account from disk

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [5]:
provider = q.IBMQ.get_provider(group='qiskit-terra', project='pulse-testing')
backend = provider.get_backend('alt_almaden')
system = q.pulse.PulseChannelSpec.from_backend(backend)
cmd_def = q.pulse.CmdDef.from_defaults(backend.defaults().cmd_def, backend.defaults().pulse_library)
measure = cmd_def.get('measure', qubits=backend.configuration().meas_map[0])

/Users/alijavadi/Desktop/ajavadia-terra/qiskit/pulse/channels/pulse_channel_spec.py:87: DeprecationWarning: The PulseChannelSpec is deprecated. Use backend.configuration() instead. The supported methods require some migrations; check out the release notes for the complete details.
  DeprecationWarning)


In [6]:
thetas = np.linspace(0, np.pi/2, 20, endpoint=False)
print(thetas)

[0.         0.07853982 0.15707963 0.23561945 0.31415927 0.39269908
 0.4712389  0.54977871 0.62831853 0.70685835 0.78539816 0.86393798
 0.9424778  1.02101761 1.09955743 1.17809725 1.25663706 1.33517688
 1.41371669 1.49225651]


In [7]:
schedules = []
for theta in thetas:
    circuit = q.QuantumCircuit(2)
    circuit.h(0)
    circuit.h(1)
    circuit.cx(0, 1)
    circuit.rz(theta, 1)
    circuit.cx(0, 1)
    circuit.h(0)
    circuit.h(1)

    decomposed_circuit = circuit.decompose()

    pulse_compiler_helper_fns.update_basis_gates_and_cmd_def(decomposed_circuit, backend, system, cmd_def)
    transpiled_circuit = q.transpile(decomposed_circuit, backend, optimization_level=1)

    schedule = q.schedule(transpiled_circuit, backend=backend, cmd_def=cmd_def)
    schedule |= measure << schedule.duration
    schedules.append(schedule)

In [8]:
shots = 1000
job = q.execute(schedules, backend=backend, shots=shots)
print('job is %s' % job.job_id())

job is 5dd6b5ab33f85e00194bfd80


/anaconda3/lib/python3.6/site-packages/ipywidgets/widgets/widget_layout.py:84: DeprecationWarning: Layout properties overflow_x and overflow_y have been deprecated and will be dropped in a future release. Please use the overflow shorthand property instead
  warnings.warn("Layout properties overflow_x and overflow_y have been deprecated and will be dropped in a future release. Please use the overflow shorthand property instead", DeprecationWarning)


In [17]:
job.status()

<JobStatus.DONE: 'job has successfully run'>

In [18]:
result = job.result()

In [19]:
from qiskit.ignis.verification.tomography.data import marginal_counts

In [20]:
counts_list = []
for i in range(len(thetas)):
    counts = marginal_counts(result.get_counts(i), meas_qubits=[0,1])    
    counts_list.append(counts)
    print(counts)

{'00': 951, '01': 30, '10': 13, '11': 6}
{'00': 954, '01': 28, '10': 15, '11': 3}
{'00': 954, '01': 23, '10': 18, '11': 5}
{'00': 937, '01': 31, '10': 16, '11': 16}
{'00': 936, '01': 27, '10': 12, '11': 25}
{'00': 908, '01': 26, '10': 26, '11': 40}
{'00': 910, '01': 23, '10': 16, '11': 51}
{'00': 866, '01': 40, '10': 21, '11': 73}
{'00': 832, '01': 43, '10': 31, '11': 94}
{'00': 830, '01': 51, '10': 24, '11': 95}
{'00': 822, '01': 29, '10': 24, '11': 125}
{'00': 800, '01': 37, '10': 21, '11': 142}
{'00': 759, '01': 42, '10': 24, '11': 175}
{'00': 725, '01': 35, '10': 26, '11': 214}
{'00': 707, '01': 39, '10': 35, '11': 219}
{'00': 683, '01': 41, '10': 25, '11': 251}
{'00': 632, '01': 45, '10': 56, '11': 267}
{'00': 628, '01': 37, '10': 41, '11': 294}
{'00': 556, '01': 42, '10': 46, '11': 356}
{'00': 508, '01': 54, '10': 47, '11': 391}


In [21]:
print(counts_list)

[{'00': 951, '01': 30, '10': 13, '11': 6}, {'00': 954, '01': 28, '10': 15, '11': 3}, {'00': 954, '01': 23, '10': 18, '11': 5}, {'00': 937, '01': 31, '10': 16, '11': 16}, {'00': 936, '01': 27, '10': 12, '11': 25}, {'00': 908, '01': 26, '10': 26, '11': 40}, {'00': 910, '01': 23, '10': 16, '11': 51}, {'00': 866, '01': 40, '10': 21, '11': 73}, {'00': 832, '01': 43, '10': 31, '11': 94}, {'00': 830, '01': 51, '10': 24, '11': 95}, {'00': 822, '01': 29, '10': 24, '11': 125}, {'00': 800, '01': 37, '10': 21, '11': 142}, {'00': 759, '01': 42, '10': 24, '11': 175}, {'00': 725, '01': 35, '10': 26, '11': 214}, {'00': 707, '01': 39, '10': 35, '11': 219}, {'00': 683, '01': 41, '10': 25, '11': 251}, {'00': 632, '01': 45, '10': 56, '11': 267}, {'00': 628, '01': 37, '10': 41, '11': 294}, {'00': 556, '01': 42, '10': 46, '11': 356}, {'00': 508, '01': 54, '10': 47, '11': 391}]
